In [2]:
nextclade = "../wdir/haplotype_info_initial_clean.csv"
outfile = "../wdir//haplotype_info_initial_clean_Sprotein.csv"
environment = "../analysis"
wdir = "../"

"../"

In [3]:
# Parameters
wdir = "/data/analyses/covid-phylogeny"
nextclade = "analysis_large/tmp/haplotypes/haplotype_info_initial_clean.csv"
outfile = "analysis_large/tmp/haplotypes/haplotype_info_initial_clean_Sprotein.csv.gz"
environment = "scripts/julia_modules/JuliaClusterAndTreeTools"


"scripts/julia_modules/JuliaClusterAndTreeTools"

In [4]:
#go to start
cd(wdir)

In [5]:
using Pkg
Pkg.activate(environment)
using DataFramesMeta
using DataFrames
using ProgressMeter
using CSV
using CodecZlib

  Activating project at `/data/analyses/covid-phylogeny/scripts/julia_modules/JuliaClusterAndTreeTools`
[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]


In [6]:
data = DataFrame(CSV.File(nextclade))
;

In [11]:
function get_S_mutations(st::AbstractString)
    parts = split(st,",")
    mtypes::Array{String,1}=[]
    wts::Array{String,1}=[]
    poss::Array{Int64,1}=[]
    subs::Array{String,1}=[]
    for s in parts
        mtype::String=""
        wt::String=""
        pos::Int=0
        sub::String=""
        ls = length(s)
        if occursin(r"^S\:",s) &&  ls > 2
            if count(i->(i==':'), s) ==2
                partsi=split(s,":")
                pos = parse(Int64,partsi[2])
                wt="-"
                sub=partsi[3]
                mtype="INS"              
            else
                ls = length(s)
                wt = s[3:3]
                sub= s[ls:ls]
                pos = parse(Int64,s[4:ls-1])
                if sub == "-"
                    mtype="DEL"
                else
                    mtype="SUB"
                end
            end
            push!(poss,pos)
            push!(wts, wt)
            push!(subs, sub)
            push!(mtypes, mtype)
        end
    end   
    return((poss,wts,subs,mtypes))
end
;

In [12]:
mTypeA::Array{String,1}=[]
wtA::Array{String,1}=[]
posA::Array{Int64,1}=[]
subA::Array{String,1}=[]
seqnameA::Array{String,1}=[]
@showprogress  for (i, m) in  enumerate(data.mutation)
    poss,wts,subs,mtypes = get_S_mutations(m)
    for j in range(1,length(poss))
        push!(mTypeA,mtypes[j])
        push!(posA,poss[j])
        push!(wtA,wts[j])
        push!(subA,subs[j])
        push!(seqnameA,data.seqName[i])
    end
end
;

Progress: 100%|█████████████████████████████████████████| Time: 0:02:1239m


In [13]:
println("Writing out long format data to ", outfile)
longdata = DataFrame(pos=posA,wt=wtA,sub=subA,mType=mTypeA,seqname=seqnameA)
CSV.write(outfile,longdata,compress=true)


Writing out long format data to analysis_large/tmp/haplotypes/haplotype_info_initial_clean_Sprotein.csv.gz


"analysis_large/tmp/haplotypes/haplotype_info_initial_clean_Sprotein.csv.gz"